<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [ ]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [ ]:
# LSST package for Butler queries
import lsst.daf.butler as dafButler

# LSST package for image display
import lsst.afw.display as afwDisplay

# Import general python packages
import time
import numpy as np
import re
import uuid
import requests
import warnings
import matplotlib.pyplot as plt
import os
import glob

# Science Pipelines imports
from lsst.daf.butler import Butler, DatasetType, CollectionType
import lsst.geom as geom
import lsst.resources
import lsst.afw.image as afwImage
from lsst.afw.image import Image, ImageF
from lsst.afw.image import Exposure, ExposureF

# Pandas tools for working with tables
import pandas
from pandas.testing import assert_frame_equal

# Plotting functions
import matplotlib.pyplot as plt

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_access_token

# PyVO packages
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Astropy
from astropy import units as u
from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.visualization import  ZScaleInterval, AsinhStretch
from astropy.wcs import WCS       
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize
from astropy.visualization.stretch import SinhStretch, LinearStretch, SqrtStretch

# Packages for warping and aligning calexp cutouts
from lsst.pipe.tasks.registerImage import RegisterConfig, RegisterTask
from PIL import Image


In [ ]:
# Set up some plotting defaults
params = {'axes.labelsize': 18,
          'font.size': 18,
          'legend.fontsize': 12,
          'xtick.major.width': 2,
          'xtick.minor.width': 1,
          'xtick.major.size': 10,
          'xtick.minor.size': 4,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 2,
          'axes.linewidth': 2,
          'axes.labelweight': 2,
          'axes.titleweight': 2,
          'ytick.major.width': 2,
          'ytick.minor.width': 1,
          'ytick.major.size': 10,
          'ytick.minor.size': 4,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [6, 6],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

# Set afw display backend to matplotlib
afwDisplay.setDefaultBackend('matplotlib')

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

plt.style.use('tableau-colorblind10')


In [ ]:
homeDir = os.getenv("HOME")
tempdir = homeDir+'/temp'
if not os.path.exists(tempdir):
    os.makedirs(tempdir)


In [ ]:
def make_image_cutout(tap_service, ra, dec, cutout_size=0.01, 
                      imtype=None, dataId=None, filename=None): #visit=None, detector=None, filename=None):
    """Make a cutout using the cutout tool
   
   Parameters
    ----------
    # cutout_size is in degrees
    # if imtype None, assumes deepCoadd. dataId must correspond to that for provided imtype
    # get difference images too (i.e. new, reference/template, and difference image). might want to 
    # (not in this function but down the road in our package of utilities to have triplet)
    # tutorial should demonstrate volume of cutouts (i.e. what i am already doing)
   Returns
    -------
    
    """

    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    
    if imtype == 'calexp':
        
        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_visit, lsst_detector, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.calexp' " + \
            "AND lsst_visit = " + str(dataId["visit"]) + " " + \
            "AND lsst_detector = " + str(dataId["detector"])
        results = tap_service.search(query)

    else:
        # Find the tract and patch that contain this point
        # this is how its done using butler:
        tract = dataId["tract"]
        patch = dataId["patch"]
        
        # add optional default band if it is not contained in the dataId
        band = dataId["band"]

        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
            "AND lsst_tract = " + str(tract) + " " + \
            "AND lsst_patch = " + str(patch) + " " + \
            "AND lsst_band = " + "'" + str(band) + "' "
        results = tap_service.search(query) 

    #Get datalink
    dataLinkUrl = results[0].getdataurl()
    f"Datalink link service url: {dataLinkUrl}"
    auth_session = service._session
    dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
    f"{dl_results.status}"

    # from_resource: creates a instance from a number of records and a Datalink Resource.
    sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                                 session=auth_session)

    sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
                 spherePoint.getDec().asDegrees()*u.deg, 
                 cutout_size* u.deg)

    if filename:
        sodaCutout = os.path.join(os.getenv('HOME'), 'temp/'+filename)
    else:
        sodaCutout = os.path.join(os.getenv('HOME'), 'temp/soda-cutout.fits')

    with open(sodaCutout, 'bw') as f:
        f.write(sq.execute_stream().read())
        
    return sodaCutout


In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry

service = get_tap_service()

### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

## 2. Section Heading

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading

In [ ]:
# first do a search of truth tables for real z>2.9 objects. search takes a really long time.
max_rec = 5000
use_center_coords = "62, -37"
ralbg = 62.
declbg = -37.
use_radius = "4.0"

query = "SELECT TOP " + str(max_rec) + " " + \
        "mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type, ts.redshift, "\
        "obj.objectId, obj.coord_ra, obj.coord_dec, obj.detect_isPrimary, obj.patch, obj.tract, " + \
        "obj.u_ap09Flux, scisql_nanojanskyToAbMag(obj.u_ap09Flux) as umag," + \
        "scisql_nanojanskyToAbMag(obj.u_ap09FluxErr) as umagErr," + \
        "scisql_nanojanskyToAbMag(obj.r_ap09Flux) as rmag, "\
        "scisql_nanojanskyToAbMag(obj.g_ap09Flux) as gmag, " + \
        "obj.u_ap09Flux/obj.u_ap09FluxErr as S2N "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec),  "\
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 "\
        "AND obj.detect_fromBlend = 0 "\
        "AND ts.redshift > 2.9 " + \
        "AND obj.g_ap09Flux/obj.g_ap09FluxErr > 10 " + \
        "AND scisql_nanojanskyToAbMag(obj.g_ap09Flux) < 24.5 " 

results = service.search(query)
#assert len(results) == max_rec
results.to_table()

In [ ]:
# skip this and test next one
#query2 = "SELECT TOP " + str(max_rec) + " " + \
#        "objectId, coord_ra, coord_dec, detect_isPrimary, patch, tract, " + \
#        "u_ap09Flux, scisql_nanojanskyToAbMag(u_ap09Flux) as umag, scisql_nanojanskyToAbMag(r_ap09Flux) as rmag, " + \
#        "scisql_nanojanskyToAbMag(g_ap09Flux) as gmag, " + \
#        "scisql_nanojanskyToAbMag(u_ap09FluxErr) as umagErr," + \
#        "u_ap09Flux/u_ap09FluxErr as S2N "\
#        "FROM dp02_dc2_catalogs.Object " + \
#        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
#        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
#        "AND detect_isPrimary = 1 AND detect_fromBlend = 0 " + \
#        "AND g_ap09Flux/g_ap09FluxErr > 10 AND r_ap09Flux/r_ap09FluxErr > 10" + \
#        "AND scisql_nanojanskyToAbMag(g_ap09Flux) < 24.5 "

#results2 = service.search(query2)
##assert len(results) == max_rec
#results2.to_table()


In [ ]:
# instead just do a radial search for all objects without matching on truth table

max_rec = 500000
query2 = "SELECT TOP " + str(max_rec) + " " + \
        "objectId, coord_ra, coord_dec, detect_isPrimary, patch, tract, " + \
        "u_ap09Flux, scisql_nanojanskyToAbMag(u_ap09Flux) as umag, scisql_nanojanskyToAbMag(r_ap09Flux) as rmag, " + \
        "scisql_nanojanskyToAbMag(g_ap09Flux) as gmag, " + \
        "scisql_nanojanskyToAbMag(u_ap09FluxErr) as umagErr," + \
        "u_ap09Flux/u_ap09FluxErr as S2N "\
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 AND detect_fromBlend = 0 " + \
        "AND g_ap09Flux/g_ap09FluxErr > 10 AND r_ap09Flux/r_ap09FluxErr > 10" + \
        "AND scisql_nanojanskyToAbMag(g_ap09Flux) < 24.5 "

results2 = service.search(query2)
#assert len(results) == max_rec
results2.to_table()

In [ ]:
#U − G >= 1.20 × (G − R) + 0.96 

# here, set umag to umag of 1-sig error before selecting colors
umag = np.zeros(len(results['umag']))
for i in range(len(umag)):

    whnondet = np.where(results['S2N'][i] < 1)[0]
    if len(whnondet) > 0:
        #print(results['umagErr'][i])
        umag[i] = results['umagErr'][i]
    else:
        umag[i] = results['umag'][i]

umag2 = np.zeros(len(results2['umag']))
for i in range(len(umag2)):

    whnondet = np.where(results2['S2N'][i] < 1)[0]
    if len(whnondet) > 0:
        #print(results2['umagErr'][i])
        umag2[i] = results2['umagErr'][i]
    else:
        umag2[i] = results2['umag'][i]

# This is Steidel+03. 
GRlim = 0.2 #1.2
GRfloor = 1.2 # 1.0
whlbg = np.where(((umag-results['gmag']) >= ((results['gmag'] - results['rmag']) + GRfloor )) & 
                 ((results['gmag'] - results['rmag']) < GRlim ) & 
                 ((results['gmag'] - results['rmag']) > 0))[0]
whlbg2 = np.where(((umag2-results2['gmag']) >= ((results2['gmag'] - results2['rmag']) + GRfloor )) & 
                  ((results2['gmag'] - results2['rmag']) < GRlim ) & 
                  ((results2['gmag'] - results2['rmag']) > 0))[0]

# plot the color-color selection box:
GmR = np.arange(0,GRlim+.01,.1) 
plt.plot(GmR,GmR + GRfloor,'--', color='k')
plt.plot([0,0],[GRfloor,3],'--', color='k')
plt.plot([GRlim,GRlim],[GRlim+GRfloor,3],'--', color='k')

color1 = results['gmag']-results['rmag']
color2 = umag-results['gmag']
#color2 = results['umag']-results['gmag']

color1_all = results2['gmag']-results2['rmag']
color2_all = results2['umag']-results2['gmag']
color2_all_fix = umag2-results2['gmag']



#plt.plot(color1_all, color2_all, '.')

#plt.plot(color1[whlbg],color2[whlbg],'o', alpha=.5,label='LBG color selections')
#plt.plot(color1_all[whlbg2],color2_all_fix[whlbg2],'o', alpha=.5,label='LBG color selections 2')
plt.plot(color1, color2, 'o',label='z~3 matched')
plt.plot(color1_all, color2_all_fix, '.',label='all objects')


plt.xlabel('G - R')
plt.ylabel('U - G')
plt.legend()
plt.show()


The next cell will plot the U-G vs G-R colors of the parent sample, and identify the high-redshift galaxy candidates based on the color selection discussed above.

Some mock galaxies at z
3 fall below the selection box, which could happen for a number of reasons (flux contamination from a neighbor, or perhaps a strong Lyman-alpha emission line that boosts the flux in the dropout filter). An exercise for the learner could be to plot instead the cutouts for the real z3 galaxies that are too blue in U-G to be selected by these criteria, to visualize a few failure modes of the color selection technique.

In [ ]:
# this cell is a test of making the cutouts for all LBG-colored objects from full search.

for i in range(len(whlbg2)):
    #print(results['redshift'][whlbg][i])
    dataId_deep = {'patch':results2['patch'][whlbg2][i], 'tract':results2['tract'][whlbg2][i], 'band':'u'}
    testu = make_image_cutout(service, results2['coord_ra'][whlbg2][i], results2['coord_dec'][whlbg2][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_u_'+str(i)+'.fits')
 
    dataId_deep = {'patch':results2['patch'][whlbg2][i], 'tract':results2['tract'][whlbg2][i], 'band':'g'}
    testg = make_image_cutout(service, results2['coord_ra'][whlbg2][i], results2['coord_dec'][whlbg2][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_g_'+str(i)+'.fits')

    dataId_deep = {'patch':results2['patch'][whlbg2][i], 'tract':results2['tract'][whlbg2][i], 'band':'r'}
    testr = make_image_cutout(service, results2['coord_ra'][whlbg2][i], results2['coord_dec'][whlbg2][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_r_'+str(i)+'.fits')

    dataId_deep = {'patch':results2['patch'][whlbg2][i], 'tract':results2['tract'][whlbg2][i], 'band':'i'}
    testi = make_image_cutout(service, results2['coord_ra'][whlbg2][i], results2['coord_dec'][whlbg2][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_i_'+str(i)+'.fits')

    dataId_deep = {'patch':results2['patch'][whlbg2][i], 'tract':results2['tract'][whlbg2][i], 'band':'z'}
    testz = make_image_cutout(service, results2['coord_ra'][whlbg2][i], results2['coord_dec'][whlbg2][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_z_'+str(i)+'.fits')

    # this gets the truth table value for the object, maybe you don't need this
    id = str(results2['objectId'][whlbg2][i])
    query3 = "SELECT scisql_nanojanskyToAbMag(obj.g_ap09Flux) as gmag, mt.id_truth_type AS mt_id_truth_type, "\
            "obj.objectId AS obj_objectId,  "\
            "obj.u_ap09Flux/obj.u_ap09FluxErr as S2N, "\
           "ts.redshift AS ts_redshift "\
            "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
            "JOIN dp02_dc2_catalogs.TruthSummary AS ts "\
            "ON mt.id_truth_type=ts.id_truth_type "\
            "JOIN dp02_dc2_catalogs.Object AS obj "\
            "ON mt.match_objectId=obj.objectId "\
            "WHERE obj.objectId="+id+" "\
            "AND ts.truth_type=1 "\
            "AND obj.detect_isPrimary=1 "
    #"mt.match_objectId AS mt_match_objectId, "\
            
    results3 = service.search(query3)
    results3.to_table()
    print(results3)
    
    fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,5, figsize=(25, 25))

    image_file = testu
    hdulist = fits.open(image_file)
    img_hdr = hdulist[1].header
    img_wcs = WCS(img_hdr)
    img_data = fits.getdata(image_file)
    ax1.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, norm='linear',
               origin='lower')
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.text(.1,.9,'u-band z='+str(results3['ts_redshift'][0]),color='white')
    image_file = testg
    img_data = fits.getdata(image_file)
    ax2.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, norm='linear',
               origin='lower')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.text(.1,.9,'g-band',color='white')
    
    image_file = testr
    img_data = fits.getdata(image_file)
    ax3.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, #norm='linear',
               origin='lower')
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.text(.1,.9,'r-band',color='white')

    image_file = testi
    img_data = fits.getdata(image_file)
    ax4.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, #norm='linear',
               origin='lower')
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.text(.9,.1,'i-band',color='white')

    image_file = testz
    img_data = fits.getdata(image_file)
    ax5.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, norm='linear',
               origin='lower')
    ax5.set_xticks([])
    ax5.set_yticks([])
    ax5.text(.1,.9,'z-band',color='white')

    plt.show()


In [ ]:
# This cell just plots the sources already known to be at z~3 from search that factors z in truth table 

for i in range(len(whlbg)):
    #print(results['redshift'][whlbg][i])
    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'u'}
    testu = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_u_'+str(i)+'.fits')
 
    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'g'}
    testg = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_g_'+str(i)+'.fits')

    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'r'}
    testr = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_r_'+str(i)+'.fits')

    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'i'}
    testi = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_i_'+str(i)+'.fits')

    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'z'}
    testz = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_z_'+str(i)+'.fits')

    # this gets the truth table value for the object, maybe you don't need this
    id = str(results['objectId'][whlbg][i])
    query3 = "SELECT scisql_nanojanskyToAbMag(obj.g_ap09Flux) as gmag, mt.id_truth_type AS mt_id_truth_type, "\
            "obj.objectId AS obj_objectId,  "\
            "obj.u_ap09Flux/obj.u_ap09FluxErr as S2N, "\
           "ts.redshift AS ts_redshift "\
            "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
            "JOIN dp02_dc2_catalogs.TruthSummary AS ts "\
            "ON mt.id_truth_type=ts.id_truth_type "\
            "JOIN dp02_dc2_catalogs.Object AS obj "\
            "ON mt.match_objectId=obj.objectId "\
            "WHERE obj.objectId="+id+" "\
            "AND ts.truth_type=1 "\
            "AND obj.detect_isPrimary=1 "
    #"mt.match_objectId AS mt_match_objectId, "\
            
    results3 = service.search(query3)
    results3.to_table()
    print(results3)
    
    fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,5, figsize=(25, 25))

    image_file = testu
    hdulist = fits.open(image_file)
    img_hdr = hdulist[1].header
    img_wcs = WCS(img_hdr)
    img_data = fits.getdata(image_file)
    #ax1.subplot(projection=img_wcs)
    ax1.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, norm='linear',
               origin='lower')
    #for ax in axes:
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.text(.1,.9,'U-band z='+str(results3['ts_redshift'][0]),color='white')
    image_file = testg
    img_data = fits.getdata(image_file)
    ax2.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, norm='linear',
               origin='lower')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.text(.1,.9,'g-band',color='white')
    
    image_file = testr
    img_data = fits.getdata(image_file)
    ax3.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, #norm='linear',
               origin='lower')
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.text(.1,.9,'r-band',color='white')

    image_file = testi
    img_data = fits.getdata(image_file)
    ax4.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, #norm='linear',
               origin='lower')
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.text(.9,.1,'i-band',color='white')

    image_file = testz
    img_data = fits.getdata(image_file)
    ax5.imshow(img_data, cmap='gray', 
               vmin=0, vmax=0.7, norm='linear',
               origin='lower')
    ax5.set_xticks([])
    ax5.set_yticks([])
    ax5.text(.1,.9,'z-band',color='white')

    plt.show()